**load in the data**

In [3]:
import json 
import os 
import pandas as pd
path = os.path.abspath("Untitled.ipynb")
print(path)
# need to add a function to grab the path more nicely 

f = open("/Users/mac/Desktop/Acc_challenge/venv/MultiWOZ2_3/data.json")
data = json.load(f)

/Users/mac/Desktop/Acc_challenge/Untitled.ipynb


**loop over the data and record an instance for everything** 
Each text instance % 0 == request, % 1 == response. - record instances 

*dialog_act*:

Each action is further broken into a smaller subset of one or more intents. 
Record in dialog_act 

**further** 
identify overall intent of conversation per unique convo_id 


In [67]:
'''
identifies the json documents where only train occurs 
'''

# each key refers to a conversation of X exchanges 
# extract the conversation goals 
conversation_list=[]
for key in data.keys(): # each conversation  
    conversation_goals_list=[]
    for item in data[key]["goal"]: 
        #if isinstance(data[key]["goal"][item], dict): # ignore list instance
           if item != "message" and item != "topic": # ignore topic and message
                if len(data[key]["goal"][item].keys()) > 0:
                    conversation_goals_list.append(item)
    conversation_list.append(conversation_goals_list)

df_convos = pd.DataFrame({"convo_id": list(data.keys()), "convo_goals": conversation_list})
# print(df_convos.head())
# print(df_convos.shape)

# returns an integer assertion if value present
def detect_target_value(row):
    if "train" in row:
        return 1 
    return 0 

# count the number of conversations 
def count_topics(row):
    return len(row)
    
df_convos["train"] = df_convos['convo_goals'].apply(detect_target_value) # looks for instance of 'train'
df_convos["num_topics"] = df_convos["convo_goals"].apply(count_topics) # looks for single instances 
df_train = df_convos.query('train == 1 & num_topics == 1') # creates a new dataframe contraining ids of only train and train only discussions 

print(df_train.shape)


(345, 4)


In [98]:
'''
Having identified where only train occurs, this cell extracts agent only utterance.
The corpus is constructed on a turn basedd appraoch and included is a turn indicator ( turn_id). 
Applying a modulous of 2, we separate out customer and agent text. 
'''
#print(data.keys())
# extract the train dictionary 
train_dict = {key: value for key, value in data.items() if key in list(df_train.convo_id)}

# drill into the log and extract the text || dialog_act || turn_id
text_list = []
dialog_list = [] 
turn_id_list = [] 
id_list = [] 
for k, v in train_dict.items():     
    for  value in v['log']:
        text_list.append(value["text"])
        dialog_list.append(value["dialog_act"].keys())
        turn_id_list.append(value["turn_id"])
        id_list.append(k)


df_utterances = pd.DataFrame({"ids": id_list, "turns": turn_id_list, "dialog_intent" : dialog_list, "text": text_list})
print(df_utterances.shape)


# def distinguishAgentCustomer(row):
#     value = row % 2
#     return value 

#df_utterances["speaker"] = df_utterances.turns.apply(distinguishAgentCustomer)
#print(df_utterances.speaker.sum())
df_agent_utterances = df_utterances[df_utterances.turns % 2 == 1]
print(df_agent_utterances.shape)
        

(3542, 4)
(1771, 4)


**intent discovery**:

Having separated out agent utterances, lets further drill into them and see if we can identify some pattern found there that can be useful in a down-the-line task. In what follows we will apply LDA to the text and see if there exists some pattern we like. 